<h1> TP transfert - Deep Learning </h1>

<hr/>

<b>N'envoyez pas votre travail par mail :</b>
- Zippez votre fichier notebook, et nommez l'archive avec votre nom
- Envoyez l'archive via la page :  https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/upload/upload.php

<hr/>

Un musée spécialisé en peintures représentant des animaux de la savane a malheureusement perdu son fichier d'inventaire qui regroupait des informations précieuses sur les 2000 oeuvres du musée ! Fort heureusement, l'informaticien toujours prévoyant avait conservé une copie des 2000 photos des oeuvres, et il vient juste de suivre une formation de Deep Learning ...

<center>
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/0.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/1.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/2.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/3.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/4.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/5.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/6.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/7.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/8.jpg" width="90px" />
<img src="https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/9.jpg" width="90px" />
</center>

Cet examen porte sur la résolution d'une tâche de classification d'images sur un jeu de données faiblement annoté. Plusieurs solutions sont envisagées  consistant à tirer parti d'annotations de classes équivalentes sur un autre dataset entièrement étiquetté (imagenet). Les deux jeux de données contiennent 2000 images réparties (également) en quatre classes (d'animaux : zèbres, gorilles, léopards, tigres). 1500 images sont utilisées pour l'entrainement, 500 pour l'évaluation. Seul le jeu de données A (imagenet) est complètement annoté, le dataset B (les photos des oeuvres du musée) ne contient que très peu d'annotations.

La partie 1 consiste à entrainer et évaluer les performances d'un modèle CNN fourni sur les deux datasets d'images. Les parties 2 et 3 consistent à mettre en oeuvre deux solutions par <i>adaptation de domaines</i> pour tenter d'améliorer les performances de classification sur le dataset faiblement annoté. Vous pouvez consacrer environ une heure à chaque partie, notées également.


<h3> Téléchargement des données </h3>

In [ ]:
!wget https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/imagenetXtrain.npy
!wget https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/imagenetYtrain.npy
!wget https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/imagenetXtest.npy
!wget https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/imagenetYtest.npy

!wget https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/awaXtrain.npy
!wget https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/awaYtrain.npy
!wget https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/awaXtest.npy
!wget https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/awaYtest.npy

<h3>Chargement des données</h3>

In [ ]:
import numpy as np
from random import shuffle
import keras
import keras.backend as K
from keras.layers import Input, Dense, Flatten, LeakyReLU
from keras.layers import AveragePooling2D, Conv2D, Activation, BatchNormalization, Dropout
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
path = './'
num_classes = 4

x_trainA = np.load(path+'imagenetXtrain.npy')
x_trainA = x_trainA / 255.
y_trainA = np.load(path+'imagenetYtrain.npy')
y_trainA = keras.utils.to_categorical(y_trainA, num_classes)

x_testA = np.load(path+'imagenetXtest.npy')
x_testA = x_testA / 255.
y_testA = np.load(path+'imagenetYtest.npy')
y_testA = keras.utils.to_categorical(y_testA, num_classes)

x_trainB = np.load(path+'awaXtrain.npy')
x_trainB = x_trainB / 255.
y_trainB = np.load(path+'awaYtrain.npy')
y_trainB = keras.utils.to_categorical(y_trainB, num_classes)

x_testB = np.load(path+'awaXtest.npy')
x_testB = x_testB / 255.
y_testB = np.load(path+'awaYtest.npy')
y_testB = keras.utils.to_categorical(y_testB, num_classes)

print('x_trainA.shape:', x_trainA.shape, 'y_trainA.shape:', y_trainA.shape)
print('x_trainB.shape:', x_trainB.shape, 'y_trainB.shape:', y_trainB.shape)

print('x_testA.shape:', x_testA.shape, 'y_testA.shape:', y_testA.shape)
print('x_testB.shape:', x_testB.shape, 'y_testB.shape:', y_testB.shape)



Les deux jeux de données contiennent 2000 images, de 64x64x3 pixels, réparties en quatre classes. 1500 images sont utilisées pour l'entrainement, 500 pour l'évaluation. Seul le jeu de données A est complètement annoté, le jeu de donnée B ne contient que très peu d'annotations, correspondantent aux 50 premiers exemples de x_trainB.

<h2> Partie 1 : Modèle convolutionnel pour la classification d'images </h2>

On définie ci-dessous une architecture convolutionnelle simple, à ne pas modifier. Cette partie vise à évaluer la performance d'un modèle CNN sur les deux dataset A et B. Entrainez ce modèle des différentes manières possibles et reportez les performances obtenues dans le tableau situé en fin de cette partie. A chaque nouvel apprentissage, utilisez les configurations suivantes :
- 30 epochs
- Taille des minibatchs à 64
- 10% des données pour validation

Tracez les courbes d'apprentissage, et commentez vos résultats.

In [ ]:
# Couches d'apprentissage de représentations par convolutions
ximg = Input(shape=(x_trainA.shape[1], x_trainA.shape[2], x_trainA.shape[3]))

l = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(ximg)
l = BatchNormalization(axis=-1)(l)
l = Activation('relu')(l)
l = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(l)
l = BatchNormalization(axis=-1)(l)
l = Activation('relu')(l)
l = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(l)
l = BatchNormalization(axis=-1)(l)
l = Activation('relu')(l)
l = AveragePooling2D((4,4))(l)
l = Conv2D(256, kernel_size=(3,3), strides=(2,2), padding='same')(l)
l = BatchNormalization(axis=-1)(l)
l = Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same')(l)
l = AveragePooling2D((2,2))(l)
l = Activation('relu')(l)
l = Dropout(0.2)(l)
feat_layer = Flatten()(l)

# conv_model(ximg) retourne la couche 'feat_layer' : 4096 dimensions pour une image 64x64x3
conv_model = Model(ximg, feat_layer)

# Couches de classification
xfeat = Input(shape=(4096,))
y = Dense(1024, activation='relu')(xfeat)
y = Dropout(0.5)(y)
y = Dense(512, activation='relu')(y)
out_layer = Dense(num_classes, activation='softmax')(y)

# classif_model(xfeat) renvoie la couche sortie du classifieur : 4 dimensions
classif_model = Model(xfeat, out_layer)

# Modèle CNN : représentation + classifier
feat = conv_model(ximg)
clf = classif_model(feat)
model = Model(ximg, clf)
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])
print(model.summary())

<h4> Entrainement du modèle sur les données A, puis évaluation sur A et B</h4>

In [ ]:
# A compléter

Commentaires :

<h4> Entrainement et évaluation du modèle sur les données B</h4>

<i>Attention à ne pas réutiliser la variable <b>model</b>, déjà entrainée sur A, il faut créer une nouvelle instance <b>model2</b> à partir des fonctions <b>conv_model()</b> et <b>classif_model()</b> définient précédemment.</i>

In [ ]:
# A compléter


Commentaires :

<h4>Finetuning sur B du modèle entrainé sur A</h4>

In [ ]:
# A compléter

Commentaires :

<table>
  <thead>
    <th></th><th>Accuracy train A</th><th>Accuracy dev A</th><th>Accuracy test A</th><th>Accuracy train B</th><th>Accuracy dev B</th><th>Accuracy test B</th>
  </thead>
  <tbody>
    <tr><th>Entrainement sur A</th><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td></tr>
    <tr><th>Entrainement sur B</th><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td></tr>
    <tr><th>Entrainement sur A <br/>+ Finetuning sur B</th><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td></tr>
  </tbody>
</table>

Commentaires :




<h2>Partie 2 : Adaptation de domaine par alignement des activations</h2>

Cette partie consiste à implémenter et évaluer un modèle inspiré de l'article <a href="https://arxiv.org/pdf/1607.01719.pdf">Deep CORAL: Correlation Alignment for Deep Domain Adaptation [Sun & Saenko, 2016] </a>, dont la figure suivante illustre le fonctionnement.

<center><img src="http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/coral16.png" width="50%" /></center>

Considérez les instructions suivantes :
- Utilisez les mêmes modules de convolutions que dans la Partie 1 afin de rester comparable. Pour cela, considérez une nouvelle instance <b>conv_model3</b> comme effectué précédemment.
- Seules les couches de convolutions sont partagées. cf https://keras.io/getting-started/functional-api-guide/#shared-layers
- Seulement deux couches denses après les convolutions (ie: fc6 et fc7), dropout entre les deux.
- Version simplifiée de la <i>CORAL loss</i> : minimisation de la distance entre les dernières couches denses (et/ou maximisation de la corrélation). Utilisez l'une des couches documentées sur la page suivante : https://keras.io/layers/merge/
- Entrainez pendant 50 epochs, avec minibatchs = 128


<h4> Définition du modèle </h4>

xA et xB sont les données des deux domaines, yA est la sortie du classifieur pour les données de xA dans le domaine source, et <i>coral</i> correspond à la métrique (distance ou corrélation) entre un minibatch d'exemples xA et xB, dans les deux domaines. Le modèle optimise les deux loss : <i>categorical_crossentropy</i> pour classer les images du domaine source ; et <i>mse</i> qui minimise (ou maximise) la métrique considérée.


In [ ]:
xA = Input(shape=(x_trainA.shape[1], x_trainA.shape[2], x_trainA.shape[3]))
xB = Input(shape=(x_trainA.shape[1], x_trainA.shape[2], x_trainA.shape[3]))

# A compléter

model4 = Model([xA,xB],[yA, coral])
model4.compile(loss=['categorical_crossentropy','mse'], optimizer=Adam(0.0001), metrics=['accuracy'])
print(model4.summary())

<h4> Entrainement du modèle d'adaptation par alignement </h4>

Selon le critère <i>coral</i> que vous optimisez, considérez le vecteur objectif ne contenant que des 1 ou que des 0..

In [ ]:
ones = np.ones((len(x_trainA),1)) # ou zeros = np.zeros((len(x_trainA),1))

# A compléter

Commentaires :

<h4>Evaluation sur les datasets A et B</h4>

In [ ]:
# A compléter

Commentaires :

<h2>Partie 3 : Adaptation de domaine par contrainte adversarial</h2>

Cette partie consiste à implémenter un modèle d'adaptation de modèle par contrainte adversarial, inspiré de l'article <a href="http://openaccess.thecvf.com/content_cvpr_2017/papers/Tzeng_Adversarial_Discriminative_Domain_CVPR_2017_paper.pdf">Adversarial Discriminative Domain Adaptation [Tzeng, 2017]</a>.

<center><img src="http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/adversadapt17.png" width="80%" /></center>

Dans la figure ci-dessus, les modules en pointillés indiquent une réutilisation de modèles déjà entrainés, tandis que les modules en lignes pleines désignent des modèles nouvellement entrainés. Pour la phase de </i>Pre-training</i>, considérez le modèle entrainé sur le dataset A dans la partie 1.

La stratégie consiste donc à :
- Faire le prétraining du modèle source sur les données source avec un critère de classification. Cette étape a été faite en partie 1 : <b>conv_model</b> et <b>classif_model</b> sont déjà entrainés.
- Ne pas réentrainer ces deux modules.
- Apprendre from scratch le modèle extracteur de caractéristiques pour les données cible à l'aide d'un discriminateur adversarial appris à discriminer entre les données source transformées par le modèle préappris et les données du domaine cible transformées par le modèle cible.  
- Construire un classifieur en empilant la couche de classification du modèle source sur l'extracteur de caractéristiques sur les données cible.



<h4> Définition du modèle discriminateur </h4>
Le discriminateur à pour objectif de distinguer les domaines de deux sources de données (A et B) à parir de représentations extraites de couches de convolutions. Ecrivez un modèle simple à deux couches cachées.

In [ ]:
# A compléter

<h4> Modèle adversarial </h4>

Ecrivez ci-dessous le modèle adversarial qui combine le module <i>Target CNN</i> et le discriminateur. <i>Target CNN</i> consistera en une nouvelle instance <b>conv_model4</b> pour extraire des représentations similaires au modèle entrainé en Partie 1 sur les images sources.

In [ ]:
# A compléter

<h4> Modèle utilisé en phase de test </h4>

Construisez ci-dessous le modèle final qui combine les modules <i>Target CNN</i>  (<b>conv_model4</b>) entrainés précédemment et <b>classif_model</b> entrainé en Partie 1.

In [ ]:
# A compléter

<h4> Entrainement du modèle adversarial</h4>

Utilisez les fonctions de création de minibatchs suivantes qui randomisent les ensembles d'apprentissage à chaque nouvel epoch.


In [ ]:
batch_size = 64
nbdata = len(x_trainA)

def get_batchA():
    global x_trainA, y_trainA
    i = 0
    while True:
        i = i + batch_size
        if i+batch_size > nbdata:
            i = 0
            lidx = list(range(nbdata))
            shuffle(lidx)
            x_trainA = x_trainA[lidx]
            y_trainA = y_trainA[lidx]
        yield x_trainA[i:i+batch_size], y_trainA[i:i+batch_size]

def get_batchB():
    global x_trainB, y_trainB
    i = 0
    while True:
        i = i + batch_size
        if i+batch_size > nbdata:
            i = 0
            lidx = list(range(nbdata))
            shuffle(lidx)
            x_trainB = x_trainB[lidx]
        yield x_trainB[i:i+batch_size]

data_genA = get_batchA()
data_genB = get_batchB()

<h4> Boucle d'apprentissage </h4>

In [ ]:
ones = np.ones((batch_size,1))
zeros = np.zeros((batch_size,1))
nb_batchs = int(len(x_trainA)/batch_size)

for epoch in range(30):
    loss = 0.0
    acc = 0.0
    for batch in range(nb_batchs):
        # get minibatchs
        xA, yA = next(data_genA)
        xB = next(data_genB)

        # A compléter

        # train discriminator

        # train adversarial model on new minibatch

        # monitoring



Commentaires :

<h4> Evaluation sur les dataset A et B </h4>

In [ ]:
# A compléter

Commentaires :

<hr/>

<b>N'envoyez pas votre travail par mail :</b>
- Zippez votre fichier notebook, et nommez l'archive avec votre nom
- Envoyez l'archive via la page :  https://pageperso.lis-lab.fr/stephane.ayache/TP_transfer/upload/upload.php
